<a href="https://colab.research.google.com/github/shubhamjha16/CME_v02/blob/master/project_CME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies

In [ ]:
!pip install flask-ngrok openai python-dotenv requests
!pip install supabase

Loading the keys

In [ ]:
import os
from google.colab import userdata

gemini_key = userdata.get("GOOGLE_API_KEY")
supabase_url = userdata.get("SUPABASE_CME_URL")
supabase_key = userdata.get("SUPABASE_CME_KEY")

os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SUPABASE_CME_URL"] = supabase_url
os.environ["SUPABASE_CME_KEY"] = supabase_key

In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

GPT SCORE FUNCTION

In [ ]:
def score_prompt(prompt, suggestion):
    try:
        model = genai.GenerativeModel('gemini-2.5-flash-lite-preview-06-17')

        strict_input = (
            "You are an expert code reviewer. Rate the following AI code suggestion from 1 to 10.\n"
            "Here is what the scores mean:\n"
            "1: Dangerous or totally incorrect\n"
            "3: Poor logic or wrong style\n"
            "5: Works but badly written or misaligned\n"
            "7: Good code with some issues\n"
            "9: Very good, minor issues\n"
            "10: Perfect\n\n"
            "Respond ONLY with the number. No explanation.\n\n"
            f"Prompt:\n{prompt}\n\nSuggestion:\n{suggestion}"
        )

        response = model.generate_content(strict_input)
        content = response.text.strip()
        score_text = content.strip().split()[0]
        score = int(''.join(filter(str.isdigit, score_text)))
        score = min(score, 10)

        accepted = score >= 6  # ✅ Add acceptance logic here
        return score, accepted

    except Exception as e:
        print("❌ Gemini Scoring Error:", e)
        return 0, False

SUPABASE WRITER FUNCTION

In [ ]:
import json

def write_to_supabase(prompt, suggestion, score, accepted, lang="", file_path=""):
    try:
        payload = {
            "prompt": str(prompt),
            "suggestion": str(suggestion),
            "score": int(score),
            "accepted": bool(accepted),
            "lang": lang or "",
            "file_path": file_path or ""
        }
        json.dumps(payload)  # Validate JSON
        response = supabase.table("prompt_log").insert(payload).execute()
        print("✅ Supabase insert success:", response)
    except Exception as e:
        print("❌ Supabase Error:", e)

In [ ]:
prompt = "Write a Python function to reverse a string."
suggestion = "def rev(s): return s[::-1]"
score = 9
accepted = True
lang = "python"
file_path = "utils/reverse.py"

write_to_supabase(prompt, suggestion, score, accepted, lang, file_path)

✅ Supabase insert success: data=[{'id': 2, 'prompt': 'Write a Python function to reverse a string.', 'suggestion': 'def rev(s): return s[::-1]', 'score': 9, 'lang': 'python', 'file_path': 'utils/reverse.py', 'accepted': True, 'created_at': '2025-06-28T17:27:42.448681'}] count=None


In [ ]:
# 1. Get score from Gemini
score, accepted = score_prompt(
  "Write a function that returns the square of a number.",
  "def square(n): return n * n"
)

# 2. Send it to Supabase
write_to_supabase(
  "Write a function that returns the square of a number.",
  "def square(n): return n * n",
  score,
  accepted,
  "python",
  "math_utils/square.py"
)

✅ Supabase insert success: data=[{'id': 3, 'prompt': 'Write a function that returns the square of a number.', 'suggestion': 'def square(n): return n * n', 'score': 10, 'lang': 'python', 'file_path': 'math_utils/square.py', 'accepted': True, 'created_at': '2025-06-28T17:34:49.158335'}] count=None


In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import google.generativeai as genai

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp

In [ ]:
!pip install supabase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
